In [1]:
import pandas as pd
from datetime import datetime

# 报表自动化
'''
指标说明：
销售额 = 单价 * 销量
客流量 = 订单ID 去重计数
客单价 = 销售额 / 客流量
本月 = 2018 年 2 月
上月 = 2018 年 1 月
去年同期 = 2017 年 2 月
'''
data = pd.read_csv('./order-1.csv',encoding='gbk',parse_dates=["成交时间"]) # parse_dates 将数据解析为时间格式
data=data.dropna()
data.head() # 预览数据
data.info() # 查看源数据类型

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3478 entries, 0 to 3477
Data columns (total 7 columns):
商品ID    3478 non-null float64
类别ID    3478 non-null float64
门店编号    3478 non-null object
单价      3478 non-null float64
销量      3478 non-null float64
成交时间    3478 non-null datetime64[ns]
订单ID    3478 non-null object
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 217.4+ KB


In [2]:
# 计算本月相关数据
this_month = data[(data['成交时间'] >= datetime(2018,2,1)) & (data['成交时间'] <= datetime(2018,2,28))]
# 销售额计算
sales_1 = (this_month['销量']*this_month['单价']).sum()
# 客流量计算
traffic_1 = this_month['订单ID'].drop_duplicates().count()
# 客单价计算
s_t_1 = sales_1 / traffic_1 
print("本月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_1,traffic_1,s_t_1))

本月销售额为：10412.78，客流量为：343，客单价为：30.36


In [5]:
# 计算上月相关指标
last_month = data[(data['成交时间'] >= datetime(2018,1,1)) & (data['成交时间'] <= datetime(2018,1,31))]
# 销售额计算
sales_2 = (last_month['销量']*last_month['单价']).sum()
# 客流量计算
traffic_2 = last_month['订单ID'].drop_duplicates().count()
# 客单价计算
s_t_2 = sales_2 / traffic_2 
print("上月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_2,traffic_2,s_t_2))

上月销售额为：9940.97，客流量为：315，客单价为：31.56


In [7]:
# 计算去年同期相关指标
same_month = data[(data['成交时间'] >= datetime(2017,2,1)) & (data['成交时间'] <= datetime(2017,2,28))]
# 销售额计算
sales_3 = (same_month['销量']*same_month['单价']).sum()
# 客流量计算
traffic_3 = same_month['订单ID'].drop_duplicates().count()
# 客单价计算
s_t_3 = sales_3 / traffic_3 
print("上月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_3,traffic_3,s_t_3))

上月销售额为：8596.31，客流量为：262，客单价为：32.81


In [9]:
# 函数
def get_month_data(data):
    sale = (data['销量']*data['单价']).sum()
    traffic =  data['订单ID'].drop_duplicates().count()
    s_t = sale / traffic
    return (sale,traffic,s_t)

this_month = data[(data['成交时间'] >= datetime(2018,2,1)) & (data['成交时间'] <= datetime(2018,2,28))]
last_month = data[(data['成交时间'] >= datetime(2018,1,1)) & (data['成交时间'] <= datetime(2018,1,31))]
same_month = data[(data['成交时间'] >= datetime(2017,2,1)) & (data['成交时间'] <= datetime(2017,2,28))]

sales_1,traffic_1,s_t_1=get_month_data(this_month)
sales_2,traffic_2,s_t_2=get_month_data(last_month)
sales_3,traffic_3,s_t_3=get_month_data(same_month)
print("本月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_1,traffic_1,s_t_1))
print("上月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_2,traffic_2,s_t_2))
print("上月销售额为：{:.2f}，客流量为：{}，客单价为：{:.2f}".format(sales_3,traffic_3,s_t_3))

本月销售额为：10412.78，客流量为：343，客单价为：30.36
上月销售额为：9940.97，客流量为：315，客单价为：31.56
上月销售额为：8596.31，客流量为：262，客单价为：32.81


In [10]:
# 将三个时间段的指标进行合并
report = pd.DataFrame([[sales_1,sales_2,sales_3],[traffic_1,traffic_2,traffic_3],[s_t_1,s_t_2,s_t_3]],columns=['本月累计','上月同期','去年同期']
                      ,index=['销售额','客流量','客单价'])
report

,本月累计,上月同期,去年同期
销售额,10412.780070,9940.972910,8596.313470
客流量,343.000000,315.000000,262.000000
客单价,30.357959,31.558644,32.810357


In [11]:
# 添加同比和环比字段
report['环比'] = report["本月累计"] / report['上月同期'] -1
report['同比'] = report['本月累计']/report['去年同期'] -1
report

,本月累计,上月同期,去年同期,环比,同比
销售额,10412.780070,9940.972910,8596.313470,0.047461,0.211308
客流量,343.000000,315.000000,262.000000,0.088889,0.309160
客单价,30.357959,31.558644,32.810357,-0.038046,-0.074745


In [12]:
report.to_csv('report.csv',encoding='utf-8-sig')

In [ ]:
# 将保存好的文件通过邮件形式发送